In [1]:
import pandas as pd
import pyarrow.parquet as pq
import numpy as np

# Read the file in chunks
def process_chunk(chunk, unique_building_blocks, unique_molecules):
    # Update the unique building blocks set
    unique_building_blocks.update(chunk['buildingblock1_smiles'].unique())
    unique_building_blocks.update(chunk['buildingblock2_smiles'].unique())
    unique_building_blocks.update(chunk['buildingblock3_smiles'].unique())
    
    # Update the unique molecules set
    unique_molecules.update(chunk['molecule_smiles'].unique())

# Load the parquet file
file_path = './train.parquet'
batch_size = 100000
parquet_file = pq.ParquetFile(file_path)

# Initialize sets to keep track of unique building blocks and molecules
unique_building_blocks = set()
unique_molecules = set()

# Iterate over the parquet file in batches
num_row_groups = parquet_file.num_row_groups

for i in range(num_row_groups):
    # Read a batch of rows
    row_group = parquet_file.read_row_group(i).to_pandas()

    if i == 0:
        print("First few rows from the first row group:")
        print(row_group.head())
    
    # Process the current chunk
    process_chunk(row_group, unique_building_blocks, unique_molecules)

# Output the total unique counts
print(f"Total number of unique building blocks: {len(unique_building_blocks)}")
print(f"Total number of unique molecules: {len(unique_molecules)}")

First few rows from the first row group:
   id                            buildingblock1_smiles buildingblock2_smiles  \
0   0  C#CC[C@@H](CC(=O)O)NC(=O)OCC1c2ccccc2-c2ccccc21  C#CCOc1ccc(CN)cc1.Cl   
1   1  C#CC[C@@H](CC(=O)O)NC(=O)OCC1c2ccccc2-c2ccccc21  C#CCOc1ccc(CN)cc1.Cl   
2   2  C#CC[C@@H](CC(=O)O)NC(=O)OCC1c2ccccc2-c2ccccc21  C#CCOc1ccc(CN)cc1.Cl   
3   3  C#CC[C@@H](CC(=O)O)NC(=O)OCC1c2ccccc2-c2ccccc21  C#CCOc1ccc(CN)cc1.Cl   
4   4  C#CC[C@@H](CC(=O)O)NC(=O)OCC1c2ccccc2-c2ccccc21  C#CCOc1ccc(CN)cc1.Cl   

     buildingblock3_smiles                                    molecule_smiles  \
0  Br.Br.NCC1CCCN1c1cccnn1  C#CCOc1ccc(CNc2nc(NCC3CCCN3c3cccnn3)nc(N[C@@H]...   
1  Br.Br.NCC1CCCN1c1cccnn1  C#CCOc1ccc(CNc2nc(NCC3CCCN3c3cccnn3)nc(N[C@@H]...   
2  Br.Br.NCC1CCCN1c1cccnn1  C#CCOc1ccc(CNc2nc(NCC3CCCN3c3cccnn3)nc(N[C@@H]...   
3        Br.NCc1cccc(Br)n1  C#CCOc1ccc(CNc2nc(NCc3cccc(Br)n3)nc(N[C@@H](CC...   
4        Br.NCc1cccc(Br)n1  C#CCOc1ccc(CNc2nc(NCc3cccc(Br)n3)nc(N[C@@H](C

# Exhaustive List of Features for Small Molecules

## Molecular Descriptors:
- Molecular weight
- Number of atoms
- Number of bonds
- Number of aromatic rings
- Number of rotatable bonds
- Topological polar surface area (TPSA)
- LogP (octanol-water partition coefficient)

## Atom-Level Features:
- Atom types (e.g., C, H, O, N, S)
- Hybridization states (sp, sp2, sp3)
- Formal charge
- Aromaticity
- Degree (number of bonds to the atom)
- Implicit and explicit hydrogen counts
- Chirality

## Bond-Level Features:
- Bond types (single, double, triple, aromatic)
- Conjugation
- Ring membership
- Stereo configuration (cis/trans)

## Graph-Based Features:
- Adjacency matrix
- Distance matrix
- Graph Laplacian

## Physicochemical Properties:
- Hydrogen bond donors and acceptors
- Molecular refractivity
- Molar volume
- Electronegativity
- Electron affinity

## Structural Fingerprints:
- MACCS keys
- Morgan fingerprints
- ECFP (Extended Connectivity Fingerprints)
- RDKIT fingerprints


In [1]:
from rdkit import Chem
from rdkit.Chem import Descriptors, rdMolDescriptors, AllChem, rdmolops
import numpy as np

class SmallMoleculeFeatureExtractor:
    def __init__(self, smiles):
        self.smiles = smiles
        self.mol = Chem.MolFromSmiles(smiles)

    def get_molecular_descriptors(self):
        descriptors = {
            'molecular_weight': Descriptors.MolWt(self.mol),
            'num_atoms': self.mol.GetNumAtoms(),
            'num_bonds': self.mol.GetNumBonds(),
            'num_aromatic_rings': rdMolDescriptors.CalcNumAromaticRings(self.mol),
            'num_rotatable_bonds': Descriptors.NumRotatableBonds(self.mol),
            'tpsa': Descriptors.TPSA(self.mol),
            'logp': Descriptors.MolLogP(self.mol)
        }
        return descriptors

    def get_atom_level_features(self):
        atom_features = []
        for atom in self.mol.GetAtoms():
            atom_features.append({
                'atom_type': atom.GetSymbol(),
                'hybridization': str(atom.GetHybridization()),
                'formal_charge': atom.GetFormalCharge(),
                'aromatic': atom.GetIsAromatic(),
                'degree': atom.GetDegree(),
                'implicit_hydrogens': atom.GetImplicitValence(),
                'explicit_hydrogens': atom.GetTotalNumHs(),
                'chirality': str(atom.GetChiralTag())
            })
        return atom_features

    def get_bond_level_features(self):
        bond_features = []
        for bond in self.mol.GetBonds():
            bond_features.append({
                'bond_type': str(bond.GetBondType()),
                'conjugation': bond.GetIsConjugated(),
                'ring': bond.IsInRing(),
                'stereo': str(bond.GetStereo())
            })
        return bond_features

    def get_graph_based_features(self):
        adj_matrix = rdmolops.GetAdjacencyMatrix(self.mol)
        dist_matrix = rdmolops.GetDistanceMatrix(self.mol)
        return {
            'adjacency_matrix': adj_matrix,
            'distance_matrix': dist_matrix,
        }

    def get_physicochemical_properties(self):
        properties = {
            'h_bond_donors': Descriptors.NumHDonors(self.mol),
            'h_bond_acceptors': Descriptors.NumHAcceptors(self.mol),
            'molecular_refractivity': Descriptors.MolMR(self.mol),
            'molar_volume': Descriptors.MolLogP(self.mol) / Descriptors.MolWt(self.mol)
        }
        return properties

    def get_structural_fingerprints(self):
        maccs_keys = AllChem.GetMACCSKeysFingerprint(self.mol)
        morgan_fp = AllChem.GetMorganFingerprintAsBitVect(self.mol, 2)
        rdk_fp = Chem.RDKFingerprint(self.mol)
        return {
            'maccs_keys': maccs_keys,
            'morgan_fp': morgan_fp,
            'rdkit_fp': rdk_fp
        }

    def extract_features(self):
        features = {
            'molecular_descriptors': self.get_molecular_descriptors(),
            'atom_level_features': self.get_atom_level_features(),
            'bond_level_features': self.get_bond_level_features(),
            'graph_based_features': self.get_graph_based_features(),
            'physicochemical_properties': self.get_physicochemical_properties(),
            'structural_fingerprints': self.get_structural_fingerprints()
        }
        return features


In [2]:
smiles = "C#CC[C@@H](CC(=O)O)NC(=O)OCC1c2ccccc2-c2ccccc21"
extractor = SmallMoleculeFeatureExtractor(smiles)
features = extractor.extract_features()
for feature, value in features.items():
    print(f"{feature}: {value}")

molecular_descriptors: {'molecular_weight': 349.3860000000001, 'num_atoms': 26, 'num_bonds': 28, 'num_aromatic_rings': 2, 'num_rotatable_bonds': 6, 'tpsa': 75.63000000000001, 'logp': 3.391700000000002}
atom_level_features: [{'atom_type': 'C', 'hybridization': 'SP', 'formal_charge': 0, 'aromatic': False, 'degree': 1, 'implicit_hydrogens': 1, 'explicit_hydrogens': 1, 'chirality': 'CHI_UNSPECIFIED'}, {'atom_type': 'C', 'hybridization': 'SP', 'formal_charge': 0, 'aromatic': False, 'degree': 2, 'implicit_hydrogens': 0, 'explicit_hydrogens': 0, 'chirality': 'CHI_UNSPECIFIED'}, {'atom_type': 'C', 'hybridization': 'SP3', 'formal_charge': 0, 'aromatic': False, 'degree': 2, 'implicit_hydrogens': 2, 'explicit_hydrogens': 2, 'chirality': 'CHI_UNSPECIFIED'}, {'atom_type': 'C', 'hybridization': 'SP3', 'formal_charge': 0, 'aromatic': False, 'degree': 3, 'implicit_hydrogens': 0, 'explicit_hydrogens': 1, 'chirality': 'CHI_TETRAHEDRAL_CW'}, {'atom_type': 'C', 'hybridization': 'SP3', 'formal_charge': 0, 

# Feature Extraction from Protein Structure PDB File

## Structural Features

### Amino Acid Composition:
- Frequency of each amino acid type in the binding site.
- Frequency of amino acid types in the entire protein.

### Secondary Structure:
- Percentage of alpha-helices, beta-sheets, and random coils in the binding site.
- Secondary structure elements around the binding site.

### Tertiary Structure:
- 3D coordinates of the binding site.
- Distance between key residues in the binding site.

### Binding Site Characteristics:
- Volume and surface area of the binding site.
- Shape descriptors (e.g., sphericity, elongation).

## Physicochemical Properties

### Hydrophobicity:
- Hydrophobic and hydrophilic residue distribution in the binding site.
- Hydrophobic surface area.

### Charge Distribution:
- Number and type of charged residues (positive and negative).
- Electrostatic potential distribution.

### Polarity:
- Number of polar residues.
- Polar surface area.

### Solvent Accessibility:
- Solvent-accessible surface area (SASA) of residues in the binding site.

### Hydrogen Bonding:
- Number of potential hydrogen bond donors and acceptors.
- Hydrogen bond network in the binding site.

### Van der Waals Interactions:
- Van der Waals interaction potential of the binding site.

## Geometric Features

### Distance Metrics:
- Pairwise distances between all residues in the binding site.
- Distance to the nearest surface residue.

### Angles and Dihedrals:
- Angles and dihedral angles between residues in the binding site.

## Chemical Environment

### Residue Environment:
- Local chemical environment of each residue (e.g., neighboring residues within a certain radius).

### Ligand Interaction Sites:
- Specific interaction sites for known ligands (if available).

## Dynamic Properties

### Flexibility:
- B-factors or temperature factors indicating residue flexibility.

### Molecular Dynamics Simulations:
- Root mean square fluctuation (RMSF) of residues in the binding site.
- Conformational changes over time.

## Topological Features

### Graph-based Features:
- Protein structure represented as a graph with nodes (residues) and edges (interactions).
- Degree centrality, betweenness centrality, and clustering coefficient of residues in the binding site.

## Energy-based Features

### Binding Energy:
- Estimated binding free energy of known ligands.
- Energy components (van der Waals, electrostatic, solvation) from docking simulations.

## Protein-Ligand Interaction Features

### Docking Scores:
- Scores from molecular docking simulations with various ligands.

### Interaction Profiles:
- Interaction fingerprints summarizing the types and strengths of interactions with ligands.

## Evolutionary Features

### Conservation:
- Sequence conservation of residues in the binding site (e.g., from multiple sequence alignment).

### Mutational Impact:
- Predicted impact of mutations on binding site residues.

## Experimental Data

### Experimental Binding Data:
- Known binding affinities (e.g., Kd, Ki, IC50) for small molecules.

## Contextual Features

### Functional Annotations:
- Biological function and pathway involvement of the protein.
- Known protein-protein interactions.

## Integration and Representation

### Feature Scaling and Normalization:
- Standardize and normalize features for input into the deep learning model.


In [ ]:
from Bio.PDB import PDBParser, PPBuilder, DSSP, NeighborSearch, is_aa
import MDAnalysis as mda
import numpy as np

class ProteinFeatureExtractor:
    def __init__(self, pdb_file, ligand_resname):
        self.pdb_file = pdb_file
        self.ligand_resname = ligand_resname
        self.structure = self.load_structure()
        self.binding_site, self.binding_site_volume = self.get_binding_site_info()

    def load_structure(self):
        # Load the PDB structure
        parser = PDBParser()
        structure = parser.get_structure('protein', self.pdb_file)
        return structure

    def get_amino_acid_composition(self):
        # Get the composition of amino acids in the protein
        amino_acids = [residue.resname for residue in self.structure.get_residues() if residue.id[0] == ' ']
        aa_counts = {aa: amino_acids.count(aa) for aa in set(amino_acids)}
        return aa_counts

    def get_secondary_structure(self):
        # Extract secondary structure information using DSSP
        model = self.structure[0]
        dssp = DSSP(model, self.pdb_file)
        secondary_structures = []
        for key in dssp.keys():
            res_id = key[1]
            sec_struc = dssp[key][2]
            secondary_structures.append((res_id, sec_struc))
        return secondary_structures

    def get_binding_site_info(self):
        # Identify binding site residues based on proximity to ligand
        u = mda.Universe(self.pdb_file)
        protein = u.select_atoms('protein')
        binding_site = protein.select_atoms(f'around 5.0 resname {self.ligand_resname}')
        volume = binding_site.total_mass()  # Simplified as an example
        binding_sites = []
        for chain in self.structure[0]:
            for residue in chain:
                if not is_aa(residue):
                    continue
                for atom in residue:
                    if atom.element == 'H':  # Skip hydrogen atoms
                        continue
                    ns = NeighborSearch(list(self.structure.get_atoms()))
                    neighbors = ns.search(atom.coord, 4.0)  # 4.0 Å distance threshold
                    for neighbor in neighbors:
                        if neighbor.get_parent().resname == self.ligand_resname:
                            binding_sites.append((residue.id, residue.resname))
                            break
        return binding_sites, volume

    def get_physicochemical_properties(self):
        # Calculate physicochemical properties of the binding site
        properties = {
            "hydrophobicity": 0,
            "charge_distribution": {"positive": 0, "negative": 0},
            "polarity": 0,
            "solvent_accessibility": 0,
            "h_bond_donors": 0,
            "h_bond_acceptors": 0
        }
        for atom in self.binding_site:
            if atom.element in ['C', 'H']:
                properties["hydrophobicity"] += 1
            if atom.element in ['O', 'N']:
                properties["polarity"] += 1
            if atom.name.startswith('N'):
                properties["charge_distribution"]["positive"] += 1
            if atom.name.startswith('O'):
                properties["charge_distribution"]["negative"] += 1
            if atom.element == 'N' and atom.name.startswith('H'):
                properties["h_bond_donors"] += 1
            if atom.element == 'O':
                properties["h_bond_acceptors"] += 1
        return properties

    def get_flexibility(self):
        # Calculate the flexibility of the protein based on B-factors
        flexibility = []
        for atom in self.structure.get_atoms():
            flexibility.append(atom.bfactor)
        return np.mean(flexibility)

    def get_distance_metrics(self):
        # Calculate distance metrics between residues in the protein
        distances = []
        for chain in self.structure.get_chains():
            for i, res1 in enumerate(chain):
                for j, res2 in enumerate(chain):
                    if i < j:
                        distances.append(res1['CA'] - res2['CA'])
        return distances

    def extract_features(self):
        # Extract various features from the protein structure
        amino_acid_composition = self.get_amino_acid_composition()
        secondary_structure = self.get_secondary_structure()
        physicochemical_properties = self.get_physicochemical_properties()
        flexibility = self.get_flexibility()
        distance_metrics = self.get_distance_metrics()

        features = {
            "amino_acid_composition": amino_acid_composition,
            "secondary_structure": secondary_structure,
            "binding_site_volume": self.binding_site_volume,
            "physicochemical_properties": physicochemical_properties,
            "flexibility": flexibility,
            "distance_metrics": distance_metrics
        }
        return features

    def aggregate_features(self, features):
        # Aggregate the extracted features into a single numpy array
        aggregated_features = []

        # Aggregate amino acid composition
        aa_composition = list(features["amino_acid_composition"].values())
        aggregated_features.extend(aa_composition)

        # Aggregate secondary structure
        sec_structure = [res[1] for res in features["secondary_structure"]]
        aggregated_features.extend(sec_structure)

        # Aggregate binding site volume
        aggregated_features.append(features["binding_site_volume"])

        # Aggregate physicochemical properties
        physico_properties = list(features["physicochemical_properties"].values())
        aggregated_features.extend(physico_properties)

        # Aggregate flexibility
        aggregated_features.append(features["flexibility"])

        # Aggregate distance metrics (average for simplification)
        avg_distance = np.mean(features["distance_metrics"]) if features["distance_metrics"] else 0
        aggregated_features.append(avg_distance)

        return np.array(aggregated_features)

    # Commented out DGL code, since it's not used
    # def extract_dgl_features(self):
        # Create a single-node graph
        # g = dgl.graph(([], []))
        # g.add_nodes(1)

        # # Extract and aggregate features
        # features = self.extract_features()
        # aggregated_features = self.aggregate_features(features)

        # # Add aggregated features to the single node
        # g.ndata['features'] = dgl.backend.tensor(aggregated_features)

        # return g

# Usage example
if __name__ == "__main__":
    pdb_file = "path/to/your/pdb_file.pdb"
    ligand_resname = "LIG"  # Replace with your ligand's residue name
    extractor = ProteinFeatureExtractor(pdb_file, ligand_resname)
    features = extractor.extract_features()
    for feature, value in features.items():
        print(f"{feature}: {value}")
    
    # Commented out DGL example usage, since it's not used
    # dgl_graph = extractor.extract_dgl_features()
    # print(dgl_graph.ndata['features'])
